In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.ensemble import IsolationForest
import json

# --- 1. Load Data ---
df = pd.read_csv('retail_price.csv')
df['month_year'] = pd.to_datetime(df['month_year'], format='%d/%m/%Y')

# --- 2. Feature Engineering and Cleaning ---

# Calculate core financial metrics
df['revenue'] = df['qty'] * df['unit_price']
df['profit'] = df['revenue'] - (df['freight_price'] + df['product_weight_g'] / 1000)
df['margin'] = df['profit'] / df['revenue']

# Calculate Competitor Price Ratios
df['price_ratio_1'] = df['unit_price'] / df['comp_1']
df['price_ratio_2'] = df['unit_price'] / df['comp_2']
df['price_ratio_3'] = df['unit_price'] / df['comp_3']

# Calculate Price Differences
df['price_diff_1'] = df['unit_price'] - df['comp_1']
df['price_diff_2'] = df['unit_price'] - df['comp_2']
df['price_diff_3'] = df['unit_price'] - df['comp_3']

# Drop original irrelevant columns and highly correlated/leaking columns
df.drop(columns=["product_id", "total_price"], inplace=True)

# Identify continuous features for outlier detection
continuous_features = [col for col in df.columns if df[col].dtype != 'object' and col not in ['month_year', 'year', 'month', 'weekday', 'weekend', 'holiday']]

# Outlier Removal using Isolation Forest (5% contamination)
model_if = IsolationForest(contamination=0.05, random_state=42)
features_for_if = df[continuous_features].fillna(df[continuous_features].mean()).to_numpy()
df['Is_Outlier'] = [1 if x == -1 else 0 for x in model_if.fit_predict(features_for_if)]
df_cleaned = df[df['Is_Outlier'] == 0].drop(columns=['Is_Outlier']).reset_index(drop=True)

# One-hot encode product category
df_encoded = pd.get_dummies(df_cleaned, columns=['product_category_name'], drop_first=True)

# --- 3. Define Features (X) and Target (y) ---

target = 'qty' # The demand target
# Exclude the target, month_year, and intermediate metrics not needed for prediction
X = df_encoded.drop(columns=[target, 'month_year', 'revenue', 'profit', 'margin'])
y = df_encoded[target]

# --- 4. Time-Based Data Split (Critical Validation Step) ---

# Train on data before May 2018, Test on data from May 2018 onwards (period of decline)
test_start_date = pd.to_datetime('2018-05-01')

X_train = X[df_encoded['month_year'] < test_start_date]
X_test = X[df_encoded['month_year'] >= test_start_date]
y_train = y[df_encoded['month_year'] < test_start_date]
y_test = y[df_encoded['month_year'] >= test_start_date]

# --- 5. Data Scaling ---

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- 6. Train and Evaluate the BEST Model (Ridge Regression) ---

# Ridge Regression Model (R^2 = 0.9425)
ridge_model = Ridge(alpha=20, random_state=42)
ridge_model.fit(X_train_scaled, y_train)

# Predictions
y_pred_ridge = ridge_model.predict(X_test_scaled)

# Evaluation
r2_ridge = r2_score(y_test, y_pred_ridge)

# --- 7. Output Final Result ---
print("--- Final Model Performance (Ridge Regression on Time-Based Split) ---")
print(f"Model: Ridge Regression (alpha=20)")
print(f"Validation: Trained on data < May 2018, Tested on data >= May 2018")
print(f"R-squared Score (on Future Data): {r2_ridge:.4f}")
print("----------------------------------------------------------------------")
print("This R^2 score validates the model's reliability for price optimization.")
print("\n" + "="*80)
print("MODEL COEFFICIENTS (for API integration)")
print("="*80)

# Extract and display coefficients
print("\nINTERCEPT:")
print(f"  intercept: {ridge_model.intercept_:.5f}")

print("\nFEATURE COEFFICIENTS:")
coefficients_dict = {}
for feature_name, coef in zip(X.columns, ridge_model.coef_):
    print(f"  {feature_name}: {coef:.5f}")
    coefficients_dict[feature_name] = round(coef, 5)

print("\n" + "="*80)
print("SCALER PARAMETERS (for API integration)")
print("="*80)
print("\nMean values:")
for i, feature_name in enumerate(X.columns):
    print(f"  {feature_name}: {scaler.mean_[i]:.5f}")

print("\nStandard Deviation values:")
for i, feature_name in enumerate(X.columns):
    print(f"  {feature_name}: {scaler.scale_[i]:.5f}")

# Store model and scaler for later use
print("\n" + "="*80)
print("PYTHON DICTIONARIES FOR API (copy this to api.py)")
print("="*80)

model_dict = {
    'intercept': float(ridge_model.intercept_),
    'coefficients': {name: float(coef) for name, coef in zip(X.columns, ridge_model.coef_)}
}

scaler_dict = {
    'means': {name: float(mean) for name, mean in zip(X.columns, scaler.mean_)},
    'scales': {name: float(scale) for name, scale in zip(X.columns, scaler.scale_)}
}

print("\nMODEL COEFFICIENTS:")
print("MODEL_COEFFICIENTS = {")
print(f"    'intercept': {ridge_model.intercept_:.5f},")
for name, coef in zip(X.columns, ridge_model.coef_):
    print(f"    '{name}': {coef:.5f},")
print("}")

print("\nSCALER PARAMETERS:")
print("SCALER_PARAMS = {")
for name, mean, scale in zip(X.columns, scaler.mean_, scaler.scale_):
    print(f"    '{name}': {{'mean': {mean:.5f}, 'std': {scale:.5f}}},")
print("}")


--- Final Model Performance (Ridge Regression on Time-Based Split) ---
Model: Ridge Regression (alpha=20)
Validation: Trained on data < May 2018, Tested on data >= May 2018
R-squared Score (on Future Data): 0.2932
----------------------------------------------------------------------
This R^2 score validates the model's reliability for price optimization.

MODEL COEFFICIENTS (for API integration)

INTERCEPT:
  intercept: 14.72709

FEATURE COEFFICIENTS:
  freight_price: -3.42217
  unit_price: -0.42640
  product_name_lenght: 0.44580
  product_description_lenght: 0.81589
  product_photos_qty: 1.78835
  product_weight_g: 0.12835
  product_score: 0.26098
  customers: 7.31556
  weekday: 0.08252
  weekend: -0.09508
  holiday: -0.19867
  month: -0.58041
  year: -0.74849
  s: 3.63656
  volume: 1.35394
  comp_1: -0.86905
  ps1: -0.85775
  fp1: -0.21628
  comp_2: 0.45438
  ps2: 1.01320
  fp2: 2.07342
  comp_3: -0.17317
  ps3: -0.80217
  fp3: 0.27260
  lag_price: 1.38927
  price_ratio_1: 0.16916
 